Нужно доделать:
* добавить обработку дублей в номерах счетчиков
* убрать слэшы в url
* избавиться от дублирующих строк

In [ ]:
# устанавливаем библиотеку scrapy для парсинга
# install Scrapy to colab
# pip install Scrapy

# устанавливаем  библиотеку для работы с Google sheets
# pip install --upgrade gspread

In [2]:
# импортируем необходимые библиотеки
import pandas as pd
import re
# для парсинга
import requests
from bs4 import BeautifulSoup
# для подсчета одинаковых значений в результате
import collections
# для работы с Google sheets
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
# для экспорта в Google sheets
from gspread_dataframe import set_with_dataframe

In [3]:
# открываем файлы
gc = gspread.authorize(GoogleCredentials.get_application_default())
worksheet = gc.open('Clients_using_analytics_TD-20668').worksheet('all_clients')
rows = worksheet.get_all_values()
# преобразуем список списков в датафрейм
clients = pd.DataFrame.from_records(rows)
# избавимся от заголовка столбца (location)
clients = clients[1:].copy()

In [4]:
# напишем парсер
# функция, собирающая в таблицу информацию об искомых скриптах на сайте
def parse_table(script, url):
    res = pd.DataFrame()

    # проверяем итерируемый скрипт на наличие искомых признаков при помощи регулярного выражения
    gtm = re.findall(r'https://www.googletagmanager.com/gtm.js', script)
    new_metrica = re.findall(r'ym\(\d{5,10}', script)
    old_metrica = re.findall(r'yaCounter\d{5,10}', script)
    web_visor = re.findall(r'webvisor:true', script)
    # условие, добавляющее информацию о скрипте, если он подошел хотя бы под один из критериев поиска
    if gtm == new_metrica == old_metrica == web_visor == []:
        pass
    else:        
        res = res.append(pd.DataFrame([[url, gtm, new_metrica, old_metrica, web_visor]], columns = ['url','gtm','new_metrica','old_metrica','web_visor']), ignore_index=True)
    return(res)

# создаем пустой ДФ для сбора результатов и указываем список сайтов для парсинга
result = pd.DataFrame()
urls = clients[0]

# применим парсер к каждому сайту из списка, заполняя ДФ с результатами
for url in urls:
    try:
        r = requests.get(url)
        soup = BeautifulSoup(r.text)
        scripts = soup.find_all('script')
        for script in scripts:
            script = str(script)
            res = parse_table(script, url)
            result = result.append(res, ignore_index=True)
    except:
        continue
# проверка
result.head()

,url,gtm,new_metrica,old_metrica,web_visor
0,https://cars.volga-rast.ru,[],[ym(62007055],[],[]
1,https://cars.volga-rast.ru,[],[ym(11332213],[],[]
2,https://cars.volga-rast.ru,[],[ym(63461152],[],[]
3,https://toyotarostov.ru,[],[ym(51363715],[],[]
4,https://toyotarostov.ru,[],[ym(23891852],[],[]


In [5]:
# сгруппируем датасет по url (клиентам)
rez = result.groupby('url')\
            .agg({'gtm':'sum', 'new_metrica':'sum', 'old_metrica':'sum',
                'web_visor':'sum'}).reset_index()
# проверка
rez.sample(5)

,url,gtm,new_metrica,old_metrica,web_visor
78,https://cars.hyundai-keyauto-rnd.ru,[https://www.googletagmanager.com/gtm.js],[ym(54580414],[],[]
260,https://toyota-kashirsky.ru,[https://www.googletagmanager.com/gtm.js],"[ym(27707151, ym(52608238]",[],[]
173,https://gacmotor.com.ru,[https://www.googletagmanager.com/gtm.js],"[ym(56563714, ym(68713000]",[],"[webvisor:true, webvisor:true]"
20,https://agat34-toyota.ru,"[https://www.googletagmanager.com/gtm.js, http...",[ym(50959181],[],[]
199,https://peugeot-alarm.ru,[https://www.googletagmanager.com/gtm.js],[ym(56567917],[],[webvisor:true]


In [6]:
# раскроем списки
rez['gtm'] = rez['gtm'].map(lambda x: ','.join(x))
rez['new_metrica'] = rez['new_metrica'].map(lambda x: ','.join(x))
rez['old_metrica'] = rez['old_metrica'].map(lambda x: ','.join(x))
rez['web_visor'] = rez['web_visor'].map(lambda x: ''.join(x))
# проверка
rez.sample(5)

,url,gtm,new_metrica,old_metrica,web_visor
137,https://cars.toyota-aurora.ru,,ym(64753930,,
4,http://cars.toyota.aspec-auto.ru,https://www.googletagmanager.com/gtm.js,ym(56789119,yaCounter35624415,
326,https://tradeins.lexus-omsk.ru,,ym(58760887,,
63,https://cars.datsun.nnmotors.ru,https://www.googletagmanager.com/gtm.js,,"yaCounter38566900,yaCounter25948823",
235,https://ton-auto.ru,https://www.googletagmanager.com/gtm.js,ym(47410633,yaCounter27191105,


In [7]:
# причешем вывод
rez['url'] = rez['url'].map(lambda x: x.replace('https://', ''))
rez['url'] = rez['url'].map(lambda x: x.replace('http://', ''))
rez['gtm'] = rez['gtm'].map(lambda x: bool(x))
rez['new_metrica'] = rez['new_metrica'].map(lambda x: x.replace('ym(', ''))
rez['old_metrica'] = rez['old_metrica'].map(lambda x: x.replace('yaCounter', ''))
rez['web_visor'] = rez['web_visor'].map(lambda x: x.replace('webvisor:true', 'Y'))
# проверка
rez.sample(5)

,url,gtm,new_metrica,old_metrica,web_visor
263,toyota-khv.ru,False,"51215327,21569422",,
234,supersale.keyauto.ru,True,,36043465,
299,toyota.by,True,"21497173,52596028",,
112,cars.lexussaratov.com,True,64773904,,
140,cars.toyota-penza.ru,False,"64760167,66191302",,


In [8]:
# посчитаем количество вебвизоров на сайте
rez['web_visors_number'] = rez['web_visor'].map(lambda x: collections.Counter(x)['Y'])
# добавим символ "'" в начале строк, чтобы Таблицы не принимали значения со списками номеров за число
rez['new_metrica'] = rez['new_metrica'].map(lambda x: r"'"+str(x))
rez['old_metrica'] = rez['old_metrica'].map(lambda x: r"'"+str(x))
# оставим только нужные столбцы
rez = rez.reindex(columns=['url','gtm','new_metrica','old_metrica','web_visors_number']).sort_values(by='url')
# проверка
rez.sample(5)

,url,gtm,new_metrica,old_metrica,web_visors_number
251,toyota-chelny.ru,False,50042938,45238389,0
94,cars.kia.yug-avto.ru,True,"43947009,37560995",,0
191,newcars.audi-chelyabinsk.ru,True,54648589,,0
209,peugeot-vladimir.ru,True,55590325,,1
103,cars.lexus-krd.ru,True,36244170,,0


In [13]:
'''
# вывод результата в Google sheets
result_sheet = gc.open('Clients_using_analytics_TD-20668').worksheet('result')
set_with_dataframe(result_sheet, rez)
'''